# Part 3: Mixed Effects Regression Models
<b>Author</b>: Sterling Cutler
<br>
<b>Date</b>: March 22, 2018

## Mixed Effects

One important consideration in using GLM's is that the model assumes the data is independent. In our auto bodily injury dataset, we are assuming the no claimant is represented in more than one claim. If a claimant had filed two different claims, then we should ask ourselves, "How might the first claim have impacted the likelihood of experiencing the second? How independent are these events?"

Some possible explanations for why these events may not be independent could include:
- first accident left claimant with impaired driving abilities (physical and/or mental)
- claimant is continuing poor driving habits that resulted in first accident
- claimant is committing multiple cases of insurance fraud

This is somewhat of a simplification, but it demonstrates the importance of understanding why models should try to capture nonindependence where it may be present. Mixed effects models handle this by accounting for random effects (or variance components).

Let's load our dataset.

In [4]:
# Load dataset and print first six rows
df <- read.csv("ABI_data.csv")
cat('Dataset shape:', dim(df))
head(df)

Dataset shape: 1339 7

ATTORNEY,CLMSEX,MARITAL,CLMINSUR,SEATBELT,NORM_CLMAGE,TARGET
1,1,2,2,1,1.1160859,3.553632
2,2,2,1,1,-0.2723103,2.388029
2,1,2,2,1,-1.7238153,-1.108663
1,1,1,2,2,-0.0198746,2.401253
2,1,4,2,1,-0.1460924,-1.980502
1,2,1,2,1,0.1694521,-1.174414


#### Creating random effects
Our dataset is assumed to have independent data, where all claims are randomly selected from the population. We will simulate random effects by introducing a state variable. How might the mean and variance of claims for a certain state related to the population of claims in our data? Any deviation between the two could be considred a random effect. While it may not seem that states "cause" claims, think about how certain elements specific to that state (e.g., population, traffic density, speed limits) may affect the number of auto accidents that occur.

In [14]:
# Add a coded state variable
states <- c('A', 'B', 'C', 'D', 'E')
n <- dim(df)[1]
df$states <- sample(states, size = n, replace=TRUE, prob=c(0.4, 0.35, 0.15, 0.09, 0.01))
head(df)

ATTORNEY,CLMSEX,MARITAL,CLMINSUR,SEATBELT,NORM_CLMAGE,TARGET,states
1,1,2,2,1,1.1160859,3.553632,A
2,2,2,1,1,-0.2723103,2.388029,A
2,1,2,2,1,-1.7238153,-1.108663,A
1,1,1,2,2,-0.0198746,2.401253,A
2,1,4,2,1,-0.1460924,-1.980502,D
1,2,1,2,1,0.1694521,-1.174414,A


## GLMM
Doc: https://cran.r-project.org/web/packages/glmm/glmm.pdf

Link: https://stats.idre.ucla.edu/other/mult-pkg/introduction-to-generalized-linear-mixed-models/

Generalized linear mixed models (or GLMMs) are an extension of linear mixed models to allow response variables from different distributions, such as binary responses. Alternatively, you could think of GLMMs as an extension of generalized linear models (e.g., logistic regression) to 
include both fixed and random effects (hence mixed models). The general form of the model (in matrix notation) is: $y=X\beta+Z\mu+\epsilon$
Where $y$ is a $Nx1$ column vector, the outcome variable; $X$ is a $Nxp$ matrix of the $p$ predictor variables; $\beta$ is a $px1$ column vector of the fixed-effects regression coefficients (the $\beta$ s); $Z$ is the $Nxq$ design matrix for the $q$ random effects (the random complement to the fixed $X$); $\mu$ is a $qX1$ vector of the random effects (the random complement to the fixed $\beta$); and $\epsilon$ is a $Nx1$ column vector of the residuals, that part of $y$ that is not explained by the model $X\beta+Z\mu$

In [ ]:
library(glmm)

glmm <- glmm(fixed="object of class 'formula'", 
             random="object of class 'formula'",
             varcomps.names="The names of the distinct variance components" +
             " in order of `varcomps.equal`",
             family.glmm="bernoulli.glmm")

summary(glmm)

## HGLM
Doc: https://cran.r-project.org/web/packages/hglm/hglm.pdf

Link: https://www.diva-portal.org/smash/get/diva2:685966/FULLTEXT02.pdf

In [ ]:
library(hglm)

hglm <- hglm(X="matrix for fixed effects", 
             y="dependent variable",
             Z="matrix for random effects",
             family = gaussian(link = identity),
             rand.family = gaussian(link = identity),
             conv=1e-6,
             maxit=50)

summary(hglm)

## LME
Doc: https://cran.r-project.org/web/packages/nlme/nlme.pdf

Link: http://www.bodowinter.com/tutorial/bw_LME_tutorial2.pdf

In [ ]:
library(nlme)

lme <- lme(fixed="two-sided linear formula of Y ~ X form", 
           data=AutoBi, 
           random="two-sided formula of form ~ x1 + ... + xn | g1/.../gm")

summary(lme)

## NLME
Doc: https://cran.r-project.org/web/packages/nlme/nlme.pdf

Link: http://lme4.r-forge.r-project.org/slides/2011-01-11-Madison/6NLMMH.pdf

In [ ]:
nlme <- nlme(model="nonlinear model formula of Y ~ X form", 
             data=AutoBi,
             fixed="two-sided linear formula of Y ~ X form", 
             random="two-sided formula of form ~ x1 + ... + xn | g1/.../gm")

summary(nlme)

## Sources
